# 1

In [1]:
import numpy as np
import pandas as pd

# read the first vector.
data = open('DS1_m_0.txt','r').readline()
d = data.split(',')
d.remove('\n')
data_0 = [float(s) for s in d]
#print(data_0)

# read the 2nd one
data = open('DS1_m_1.txt','r').readline()
d = data.split(',')
d.remove('\n')
data_1 = [float(s) for s in d]
#print(data_1)

In [2]:
# we will write a function that calculate the best fit
def c(a):
    p = 0
    n = 0
    for i in a:
        if (i > 0):
            p = p + 1
        else:
            n = n + 1
    return positive, negative

# lecture 8 formulas,
def best_fit(a,b):
    
    
    accuracy = (c(a)[0] + c(b)[1])/1200
    precision = (c(a)[0])/(c(a)[0] + c(b)[1])
    recall = (c(a)[0])/(c(a)[0] + c(b)[0])
    F = 2*precision/(precision + recall)
    return accuracy, precision, recall, F




In [3]:
cov = []
with open('DS1_Cov.txt','r') as file:
    for line in file:
        #read the row
        r = line.split(',')
        r.remove('\n')
        row = [float(s) for s in r]
        #add it to the matrix
        cov = cov + [row]


In [4]:
negative = np.random.multivariate_normal(data_0,cov,2000)
positive = np.random.multivariate_normal(data_1,cov,2000)

In [5]:
import random
#Get the test and training data.
random.shuffle(negative)
Train_0 = negative[:1400]
Test_0  = negative[1400:]

random.shuffle(positive)
Train_1 = positive[:1400]
Test_1  = positive[1400:]

DS1 = np.concatenate((Test_0, Test_1), axis=0)

random.shuffle(DS1)
file = open('DS1.txt', 'w')
for x in DS1:
    file.write("%s\n" % x)

# 2

In [6]:
# First maximization, we calcualte the means (pi is 1/2 obviously)

M0 = np.mean(Train_0, axis=0)
M1 = np.mean(Train_1, axis=0)

#print(Train_1)
#print("---------------")
#print(Train_0)
# now the covariance maximization.
# first calculate s1/N
s1 = np.zeros((20,20))
for x in Train_0:
    x = np.subtract(x,M0)
    b = np.array(x).T
    A = b.reshape(-1,1)
    B = np.dot(A,[x] )
    s1 = np.add(s1, B)
s1 = np.divide(s1,2800)

# now s2/N
s2 = np.zeros((20,20))
for x in Train_1:
    x = np.subtract(x,M1)
    b = np.array(x).T
    A = b.reshape(-1,1)
    B = np.dot(A,[x] )
    s2 = np.add(s2, B)
s2 = np.divide(s2,2800)

# getting the covariance
COV = np.add(s1,s2)
#print(M1)
#print(M0)

In [7]:
# now to get the parameters W0 and W
# we start first with W
E_1 = np.linalg.inv(COV)
W = np.dot(E_1, ( np.subtract(M0,M1) ) )

# now for W0
M0_t = M0.reshape(-1,1)
A = np.dot(M0,E_1)
A = np.dot(A,M0_t)
A = np.dot(-1/2,A)

M1_t = M1.reshape(-1,1)
B = np.dot(M1,E_1)
B = np.dot(B,M1_t)
B = np.dot(1/2,B)

W0 = A[0]+B[0]



In [8]:
## now for testing !!

def classify(W,W0,x):
    a = np.dot(W.T,x) + W0
    res = 1/(1+np.exp(-a))
    if res > 0.5:
        return 1
    else:
        return 2
    
# Performance test
DS1, Test_0, Test_1

p = np.matmul(Test_0,W) + W0
n = np.matmul(Test_1,W) + W0


def c(a):
    p = 0
    n = 0
    for i in a:
        if (i > 0):
            p = p + 1
        else:
            n = n + 1
    return positive, negative

score = 0
for t in DS1:
    v = classify(W, W0, t)
    if v == 1 :
        score = score + 1
    elif v == 2:
        score = score +1 

    else:
        continue


accuracy = (c(p)[0] + c(n)[1])/1200
precision = (c(p)[0])/(c(p)[0] + c(p)[1])
recall = (c(p)[0])/(c(p)[0] + c(n)[0])



print("The accuracy of this classifier is " ,accuracy)
print("The precision of this classifier is " ,precision) 
print("The recall of this classifier is " ,recall) 
print("The F-measure of this classifier is " ,2*precision/(precision + recall))


score/1200

The accuracy of this classifier is  0.9483333333333334
The precision of this classifier is  0.9566666666666667
The recall of this classifier is  0.940983606557377
The F-measure of this classifier is  1.0082644628099173


1.0

## 3 K-NN


In [14]:
import math
import operator
# now we write a function that calculate euclidien distance
def Dis(x1,x2):
    res = 0
    for i in range(len(x1)):
        res += pow((x1[i] - x2[i]), 2)
    return math.sqrt(res)


def knearestneighbourhood(train,point,k):
    distances = []
    length = len(train)
    for x in range(0,length):
        dist = Dis(point,X[x])
        if (x < length/2):
            distances.append((dist,1))
        else:
            distances.append((dist,0))
    distances = sorted(distances)[:k]
    frequency_p = 0
    frequency_n = 0 
    
    for dist in distances:
        if dist[1] == 0:
            frequency_n += 1
        elif dist[1] == 1:
            frequency_p += 1
    return 0 if frequency_n>frequency_p else 1
def getresultk(test,train,k):
    out = []
    for i in test:
        out.append(knearestneighbourhood(train,i,k))
    return out


def best_fit_KNN(positive_values,negative_values):
    
    highest_f_measure = -1
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > highest_f_measure):
            highest_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*precision/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index


TrainingSet = np.concatenate((Train_0, Train_1), axis=0)

random.shuffle(TrainingSet)
def best_fit_KNN(positive_values,negative_values):
    
    highest_f_measure = -1
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > highest_f_measure):
            highest_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*precision/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index

X=list(TrainingSet)
positive_values = []
negative_values = []

# we try diff values for k
for k in range(1,8):
    sample1 = getresultk(Test_0, Train_0,k)
    sample2 = getresultk(Test_1, Train_1,k)
    positive_values.append(count(sample1))
    negative_values.append(count(sample2))

c,d,e,f,k = best_fit_KNN(positive_values,positive_values)
print("The accuracy of this classifier is ",c)
print("The precision of this classifier is ",d)
print("The recall of this classifier is ",e)
print("The F-measure of this classifier is ",f)
print("The best k value is:",k)

    

The accuracy of this classifier is  0.5
The precision of this classifier is  0.335
The recall of this classifier is  0.5
The F-measure of this classifier is  0.8023952095808384
The best k value is: 3


# 4

In [15]:
import numpy as np
probability = [0.1,0.42,0.48]
def param(Cov,M0,M1):
    E_1 = np.linalg.inv(Cov)
    W = np.dot(E_1, ( np.subtract(M0,M1) ) )
    # now for W0
    M0_t = M0.reshape(-1,1)
    A = np.dot(M0,E_1)
    A = np.dot(A,M0_t)
    A = np.dot(-1/2,A)

    M1_t = M1.reshape(-1,1)
    B = np.dot(M1,E_1)
    B = np.dot(B,M1_t)
    B = np.dot(1/2,B)
    W0 = A[0]+B[0]
    
    return W,W0

def split(data):
    np.random.shuffle(data)
    test = []
    train = []
    for row in data[0:600]:
        test.append(row)
    
    for row in data[600:2001]:
        train.append(row)
        
    return test,train

def classify(W,W0,x):
    a = np.dot(W.T,x) + W0
    res = 1/(1+np.exp(-a))
    if res > 0.5:
        return 1
    else:
        return 2
    

ds2c1m1 = np.genfromtxt("DS2_c1_m1.txt", delimiter=',')
ds2c1m2 = np.genfromtxt("DS2_c1_m2.txt", delimiter=',')
ds2c1m3 = np.genfromtxt("DS2_c1_m3.txt", delimiter=',')

ds2c2m1 = np.genfromtxt("DS2_c2_m1.txt", delimiter=',')
ds2c2m2 = np.genfromtxt("DS2_c2_m2.txt", delimiter=',')
ds2c2m3 = np.genfromtxt("DS2_c2_m3.txt", delimiter=',')

ds2cov1 = np.genfromtxt("DS2_Cov1.txt", delimiter=',')
ds2cov2 = np.genfromtxt("DS2_Cov2.txt", delimiter=',')
ds2cov3 = np.genfromtxt("DS2_Cov3.txt", delimiter=',')

ds2c1m1=ds2c1m1[:20]
ds2c1m2=ds2c1m2[:20]
ds2c1m3=ds2c1m3[:20]
ds2c2m1=ds2c2m1[:20]
ds2c2m2=ds2c2m2[:20]
ds2c2m3=ds2c2m3[:20]
ds2cov1=ds2cov1[:,:20]
ds2cov2=ds2cov2[:,:20]
ds2cov3=ds2cov3[:,:20]



sampling1 = np.random.multinomial(2000,probability)
sampling2 = np.random.multinomial(2000,probability)

class1_m1 = np.random.multivariate_normal(ds2c1m1,ds2cov1, sampling1[0])
class1_m2 = np.random.multivariate_normal(ds2c1m2,ds2cov2, sampling1[1])
class1_m3 = np.random.multivariate_normal(ds2c1m3,ds2cov3, sampling1[2])
class2_m1 = np.random.multivariate_normal(ds2c1m1,ds2cov1, sampling1[0])
class2_m2 = np.random.multivariate_normal(ds2c1m2,ds2cov2, sampling1[1])
class2_m3 = np.random.multivariate_normal(ds2c1m3,ds2cov3, sampling1[2])

# Append list to form a class.
positive2 = list(class1_m1)
positive2.extend(list(class1_m2))
positive2.extend(list(class1_m3))

negative2 = list(class2_m1)
negative2.extend(list(class2_m2))
negative2.extend(list(class2_m3))

q4_test1, q4_train1 = split(positive2)
q4_test2, q4_train2 = split(negative2)

TrainingSet = np.concatenate((q4_train1, q4_train2), axis=0)
Test = np.concatenate((q4_test1, q4_test2), axis=0)

# 5

In [16]:
# now repeating the experiments 2 and 3.


M0 = np.mean(q4_train1, axis=0)
M1 = np.mean(q4_train2, axis=0)

s1 = np.zeros((20,20))
for x in q4_train1:
    x = np.subtract(x,M0)
    b = np.array(x).T
    A = b.reshape(-1,1)
    B = np.dot(A,[x] )
    s1 = np.add(s1, B)
s1 = np.divide(s1,2800)

# now s2/N
s2 = np.zeros((20,20))
for x in q4_train2:
    x = np.subtract(x,M1)
    b = np.array(x).T
    A = b.reshape(-1,1)
    B = np.dot(A,[x] )
    s2 = np.add(s2, B)
s2 = np.divide(s2,2800)
def Dis(x1,x2):
    res = 0
    for i in range(len(x1)):
        res += pow((x1[i] - x2[i]), 2)
    return math.sqrt(res)

def knearestneighbourhood(train,point,k):
    distances = []
    length = len(train)
    for x in range(0,length):
        dist = Dis(point,train[x])
        if (x < length/2):
            distances.append((dist,1))
        else:
            distances.append((dist,0))
    distances = sorted(distances)[:k]
    frequency_p = 0
    frequency_n = 0 
    
    for dist in distances:
        if dist[1] == 0:
            frequency_n += 1
        elif dist[1] == 1:
            frequency_p += 1
    return 0 if frequency_n>frequency_p else 1
def getresultk(test,train,k):
    out = []
    for i in test:
        out.append(knearestneighbourhood(train,i,k))
    return out

# getting the covariance
COV = np.add(s1,s2)
#print(M0,M1)
W,W0 = param(COV, M0, M1 )

a = np.matmul(q4_test1,W) + W0
b = np.matmul(q4_test2,W) + W0

c,d,e,f = best_fit(a,b)

print("The accuracy of this classifier is ",c)
print("The precision of this classifier is ",d)
print("The recall of this classifier is ",e)
print("The F-measure of this classifier is ",f)

The accuracy of this classifier is  0.4841666666666667
The precision of this classifier is  0.4819277108433735
The recall of this classifier is  0.4835924006908463
The F-measure of this classifier is  0.9982758620689655


In [17]:
c =[]
d =[]
e =[]
f =[]
pos = []
neg = []

X=list(q4_train1)
positive_values = []
negative_values = []

# we try diff values for k
for k in range(1,8):
    sample1 = getresultk(q4_test1, q4_train1,k)
    sample2 = getresultk(q4_test2, q4_train2,k)
    positive_values.append(count(sample1))
    negative_values.append(count(sample2))
    

def best_fit_KNN(positive_values,negative_values):
    
    highest_f_measure = -1
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > highest_f_measure):
            highest_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*precision/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index

In [18]:
def best_fit_KNN(positive_values,negative_values):
    
    highest_f_measure = -1
    index = 0
    for i in range(0,len(positive_values)):
        accuracy = (positive_values[i][0] + negative_values[i][1])/1200
        precision = (positive_values[i][0])/(positive_values[i][0] + positive_values[i][1])
        recall = (positive_values[i][0])/(positive_values[i][0] + negative_values[i][0])
        F_measure = 2*precision/(precision + recall)
        if (F_measure > highest_f_measure):
            highest_f_measure = F_measure
            index = i
    accuracy = (positive_values[index][0] + negative_values[index][1])/1200
    precision = (positive_values[index][0])/(positive_values[index][0] + positive_values[index][1])
    recall = (positive_values[index][0])/(positive_values[index][0] + negative_values[index][0])
    F_measure = 2*precision/(precision + recall)
    
    return accuracy,precision,recall,F_measure,index



c,d,e,f,k = best_fit_KNN(positive_values,positive_values)
print("The accuracy of this classifier is ",c)
print("The precision of this classifier is ",d)
print("The recall of this classifier is ",e)
print("The F-measure of this classifier is ",f)
print("The best k value is:",k)


The accuracy of this classifier is  0.5
The precision of this classifier is  0.765
The recall of this classifier is  0.5
The F-measure of this classifier is  1.209486166007905
The best k value is: 1
